In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#library(fUnitRoots)
library(forecast)
library(TSstudio)
library(tseries)

<a id="3"></a> <br>
## Parte 1: Simulacion procesos AR (ejercicio final presentación pdf)

<a id="4"></a> <br>
### Gráfico 1 Simulacion de un proceso AR(1) con phi=0.6

In [5]:
layout(matrix(c(1,1,2,3), 2, 2, byrow = TRUE))
AR_1=arima.sim(list(order=c(1,0,0), ar=.6), n=500, sd = sqrt(0.2))  
plot(AR_1, ylab=" ", main=(expression(AR(1)~~~phi==.6)))


al ser un phi menor a 1 su proceso debería ser estacionario

In [10]:
par(mfrow=c(1,2))
acf(AR_1, main="ACF-Autocorrelación Simple",ylim=c(-.5,.5),ci.col="blue",ylab="")
pacf(AR_1,main="PACF-Autocorrelación Parcial",ylim=c(-.5,.5),ci.col="blue")


La PACF podría sugerir un AR(9), sin embargo, por conveniencia del modelo sería más apropiado debería tomarse con un A(1).

<a id="4"></a> <br>
### Gráfico 2 Simulacion de un proceso AR(1) con phi_1=-0.8

In [7]:
layout(matrix(c(1,1,2,3), 2, 2, byrow = TRUE))
AR_2=arima.sim(list(order=c(1,0,0), ar=-.8), n=500, sd = sqrt(0.2))  
plot(AR_2, ylab=" ", main=(expression(AR(1)~~~phi==-.8)))

al ser un phi menor a 1 su proceso debería ser estacionario

In [9]:
par(mfrow=c(1,2))
acf(AR_2, main="ACF-Autocorrelación Simple",ylim=c(-.5,.5),ci.col="blue",ylab="")
pacf(AR_2,main="PACF-Autocorrelación Parcial",ylim=c(-.5,.5),ci.col="blue")


La PACF sugeriría un AR(1).

<a id="4"></a> <br>
### Gráfico 3 Simulacion de un proceso AR(2) con phi_1=0.3 y phi_2=-0.6

In [12]:
#set.seed(12345)
ar2 <- arima.sim(model = list(order(2,0,0), ar = c(0.3, -0.6), ma = c(0,0)), n = 500)
plot(ar2)

In [13]:
par(mfrow=c(1,2))
acf(ar2, main="ACF-Autocorrelación Simple",ylim=c(-.5,.5),ci.col="blue",ylab="")
pacf(ar2,main="PACF-Autocorrelación Parcial",ylim=c(-.5,.5),ci.col="blue")

La PACF sugeriría un AR(2). Esto con 0 < ∅1 < 1 y -1 < ∅2 < 0.

<a id="4"></a> <br>
### Análisis de los datos del archivo rend.csv

#### (frequency=12) : Utilizacion de Datos mensuales

In [14]:
rend=read.csv("../input/rend-ej1/rend.csv", header=TRUE, sep=",")
head(rend)
rend=rend[,-1]
rend=ts(rend, start=c(1976,1), freq=12)
ts_plot(rend, title = "AR (3) Process",Ytitle = "Value", Xtitle = "Index")
par(mfrow=c(1,2))
acf(rend)
pacf(rend)

La PACF sugeriría un AR(3).

### Modelo AR

Se inicia con un AR(3), esto a partir del PACF recién evaluado

In [15]:
m0<-Arima(rend,order = c(3,0,0)) # con la visualización corta en 3
m0
m00<-arima(rend, order = c(3,0,0))
m00
sigma=m0$sigma2
tsdiag(m0)
shapiro.test(m0$residuals)

la prueba de Ljung-Box muestra el modelo tiene un buen ajuste porque presenta p-values alejados del nivel de significancia, es decir no conservan raíz unitaria y se puede decir que garantiza la estacionariedad. Así mismo sus residuales presentan un comportamiento de ruido blanco

### Gráficos de adecuación al modelo

#### Modelo diferenciado

In [16]:
rend=read.csv("../input/rend-ej1/rend.csv", header=TRUE, sep=",")
head(rend)
rend=rend[,-1]
rend=ts(rend, start=c(1976,1), freq=12)

In [17]:
##Para modelos ARMA, debemos hacer estacionaria la serie, por ello, sacamos la primera diferencia de la misma.
rend.dif=diff(rend)
adf.test(rend.dif)   ##Ahora ya debería ser estacionaria
plot(rend.dif)
##Se esperaría que ahora la serie ya sea estacionaria, esto sólo aplicando una diferencia a la misma. 

Después de aplicar una diferenciación se hace la prueba Dicky-Fuller, de esta se encuentra que la serie ahora no es estacionaria

Ahora se evalúa su PACF para entender su nivel AR

In [19]:
par(mfrow=c(1,2))
acf(rend.dif)
pacf(rend.dif)

Se muestra que un modelo AR(12) sería el más conveniente al aplicar a la diferenciación hecha sobre la serie original de rendimientos

In [22]:
##Modelamos así, la AR()
mod=ar(rend.dif,method='mle')  #Función para derivar únicamente un modelo tipo AR(p)
mod$order

#### Modelación para obtener parámetros y evaluar modelo

#### Serie con ajuste Arima

In [23]:
rend.diff.ts=ts(rend.dif,start=c(1976,1), frequency=12) #generamos el indexado de fecha mensual
m1= Arima(rend.diff.ts,order=c(12,1,0), method = "ML") #Se aplica el método de maximum likelihood
m1
tsdiag(m1)

In [25]:
m1b=Arima(rend, order=c(12,1,0), method="ML", lambda=TRUE)  #Aplicamos la función de regresión Arima a serie original, especificando la transformación de Box-Cox
m1b
tsdiag(m1b)

Al aplicar una serie ARIMA a este orden para las dos series (original y diferenciada) con el método de “maximum likelihood”. En ambos casos se logran series estacionarias con p-values alejados de la zona de rechazo y con comportamiento de ruido blanco. Ahora bien, el más ajustado es el AR(12) original, esto tal como se muestra a continuación:

In [26]:
m1c=auto.arima(rend, method = "ML")
m1c
##A través de auto.arima, que es una función que busca la combinación que aminora el AIC y BIC

En este caso el mejor modelo desde la revisión del AIC es el ARIMA(4,0,1)

Ahora se prueba con el ajuste a una serie logarítmica

In [27]:
da=read.table("../input/rend-ej1/rend.csv",header=T)
head(da)

#### Transformación Serie logarítmica

In [48]:
library(tseries)
rend=log(da[,1])  ##transformación log para estabilizar la tendencia
adf.test(rend)
rend.dif=diff(rend)
##la hace estacionaria
adf.test(rend.dif)
rend.diff.ts=ts(rend.dif,start=c(1976,1), frequency=12)
plot(rend.diff.ts)

Se comprueba que la serie log no es estacionaria, esto dado su p-value menor al nivel de significancia

In [29]:
par(mfrow=c(1,2))
acf(rend.dif)
pacf(rend.dif)

In [23]:
mod1=ar(rend.dif,method='mle')
mod1$order
##Vemos ahora, que el niveo AR es de 2.

In [26]:
rend.ts=ts(rend,start=c(1976,1), frequency=12) #generamos el indexado de fecha con frecuencia mensual
m3dif= Arima(rend.ts,order=c(2,1,0), method = "ML", lambda="auto") #ae utiliza el método de Maximum Likelihood
#ya es estacionaria, por tanto (n,1,0)
m3dif
tsdiag(m3dif)

En este caso se puede ver como la serie no es estacionaria, esto por medio de la prueba de Ljung-box donde los p-value estan fuera de la zona de rechazo, esto indica que la transformación logarítimica anterior no logra volver la serie estacionaria 

In [27]:
# coeficiente integrado m es la cantidad de veces que tuve que diferenciar en (n,m,0)
m1b=Arima(rend.dif, order=c(2,1,0), method="ML", lambda=TRUE)  #Aplicamos la función de regresión Arima a serie original, especificando la transformación de Box-Cox
m1b
tsdiag(m1b)

Para este caso se aplica igualmente la prueba de Box-Cox, sin embargo, podemos definir que el modelo anterior a esta serie rend.diff.ts (m1b) tiene menor ajuste, esto por medio del AIC, en su caso debido a que es mayor que el de la serie (m3dif)

### Pronóstico 12 pasos adelante

In [30]:
library(TSstudio)
library(devtools)

In [63]:
rend=read.csv("../input/rend-ej1/rend.csv", header=TRUE, sep=",")
rend=rend[,-1]
rend=ts(rend,start=c(1976,1), frequency=12) #generamos el indexado de fecha con frecuencia mensual
plot(rend) #es la serie no estacionaria. Para el modelado lineal, no tiene que cumplir la condición de estacionariedad.

In [64]:
#Partimos la serie en entrenamiento y prueba
rend_par <- ts_split(rend, sample.out = 12)  ##Funcion muy util para partir facilmente la serie.
train <- rend_par$train
test <- rend_par$test
##Aplicamos modelo tslm()
library(forecast)
md <- tslm(train ~ trend +  trend^2 + season, lambda= "auto")  #Es una regresion lineal con series tiempo, descompone componentes y los incirpora en modelo.
#la opción de lambda, aplica una trasnformación de box cox automática para estabilizar la tendencia, regresando los resultados a las unidades originales.
summary(md)

In [65]:
#Pronóstico:
fc <- forecast(md, h = 12)
##Métricos de desempeño de pronóstico
test_forecast(actual = rend,
              forecast.obj = fc,
              test = test)

In [66]:
pron<-forecast(m1, h=12) # ventana
pron

In [68]:
##Se construye la especificacion de los modelos de interes a probar:
methods <- list(ets1 = list(method = "ets",
                            method_arg = list(opt.crit = "lik", lambda = "auto"),#opt.crit: optimization criterion: Log-likelihood es "lik", default.
                            notes = "ETS model with opt.crit = lik"),
                ets2 = list(method = "ets",
                            method_arg = list(opt.crit = "amse", lambda = "auto"),    #creterio optimizaci??n con Average mean square of error.
                            notes = "ETS model with opt.crit = amse"),
                arima1 = list(method = "arima",
                              method_arg = list(order = c(2,1,0)),
                              notes = "ARIMA(2,0,0)"),
                arima2 = list(method = "arima",
                              method_arg = list(order = c(3,1,0)),
                              notes = "ARIMA(3,1,0)"),
                hw = list(method = "HoltWinters",
                          method_arg = NULL,
                          notes = "HoltWinters Model"),
                tslm = list(method = "tslm",
                            method_arg = list(formula = input ~ trend + season, lambda= "auto"),
                            notes = "tslm model with trend and seasonal components"))

train_method = list(partitions = 6,
                    sample.out = 12,
                    space = 3)

md <- train_model(input = rend,
                  methods = methods,
                  train_method = train_method,
                  horizon = 12,
                  error = "RMSE")

In [69]:
library(fastmap)
update.packages("fastmap")
plot_error(md, error = "RMSE", palette = "Set1")

In [56]:
# Retrieve the models leaderboard
md$leaderboard